In [6]:
import itertools


In [7]:
items = [1, 2, 3]
for item in itertools.permutations(items, 3):
    print(item)


(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


In [8]:
for item in itertools.combinations(items, 2):
    print(item)


(1, 2)
(1, 3)
(2, 3)


In [9]:
for item in itertools.combinations_with_replacement(items, 2):
    print(item)


(1, 1)
(1, 2)
(1, 3)
(2, 2)
(2, 3)
(3, 3)


In [10]:
ab = ['a', 'b']
cd = ['c', 'd']
# Produce the product of two arrays
for item in itertools.product(ab, cd):
    print(item)


('a', 'c')
('a', 'd')
('b', 'c')
('b', 'd')


In [31]:

# 1. Defining a function that takes params and return result+params

def calc(keep_stock_threshold, buy_change_threshold):
    """
    :param keep_stock_threshold: 
    :param buy_change_threshold: 
    :return:  
    """

    # Initialize an instance of Strategy2
    import import_ipynb
    from strategy_2 import TradeStrategy2
    trade_strategy2 = TradeStrategy2()
    
    # Use class method to set keep_stock_threshold
    TradeStrategy2.set_keep_stock_threshold(keep_stock_threshold)
    
    # Use static method to set buy_change_threshold
    TradeStrategy2.set_buy_change_threshold(buy_change_threshold)
    
    # Create Trade Days (MasterCard)
    from abupy import ABuSymbolPd, EMarketDataSplitMode
    # List 2 years of MasterCard closing price to list()
    price_array = ABuSymbolPd.make_kl_df('MA', EMarketDataSplitMode.E_DATA_SPLIT_SE).close.tolist()
    date_array = ABuSymbolPd.make_kl_df('MA', EMarketDataSplitMode.E_DATA_SPLIT_SE).date.tolist()
    price_array[-5:], date_array[-5:]
    
    from tradedays import StockTradeDays
    trade_days = StockTradeDays(price_array, "", date_array)
    # print('trade_days has {} days'.format(len(trade_days)))
    # print('Last trading day: {}'.format(trade_days[-1]))

    # Backtest
    from trade_loopback import TradeLoopBack
    trade_loop_back = TradeLoopBack(trade_days, trade_strategy2)
    trade_loop_back.execute_trade()
    
    # Calculate profit after back test
    from functools import reduce
    profit = 0.0 if len(trade_loop_back.profit_array) == 0 else \
        reduce(lambda a, b: a + b, trade_loop_back.profit_array)
    
    # Return profit and the two input params
    return profit, keep_stock_threshold, buy_change_threshold

calc(5, -0.02)


(0.1370000000000001, 5, -0.02)

In [42]:

# range: buy and then hold 2~30 days, step is 2
keep_stock_days_list = range(2, 30, 2)
print('keep_stock Array: {}'.format(list(keep_stock_days_list)))
# range: 'buy_change' from -0.05 to -0.15, aka down 5% to 15%
buy_change_list = [buy_change / 100.0 for buy_change in range(-5, -16, -1)]
print('buy_change Array: {}'.format(buy_change_list))


keep_stock Array: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
buy_change Array: [-0.05, -0.06, -0.07, -0.08, -0.09, -0.1, -0.11, -0.12, -0.13, -0.14, -0.15]


In [43]:
result = []
for keep_stock_threshold, buy_change_threshold in itertools.product(
    keep_stock_days_list, buy_change_list):
    result.append(calc(keep_stock_threshold, buy_change_threshold))

print('Cartesian product produced {} results'.format(len(result)))


HTTPConnectionPool(host='gp.baidu.com', port=80): Max retries exceeded with url: /stocks/stockkline?from=android&os_ver=21&format=json&vv=3.3.0&uid=&BDUSS=&cuid=9jg3uwqut4fbpo0b3f9vg0pamrgj2guetjln86b1&channel=default_channel&device=OPPOR9&logid=1559200886&actionid=1559200820&device_net_type=wifi&period=day&stock_code=usMA&fq_type=front (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1076ccd30>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Traceback (most recent call last):
  File "/Users/XujieSong/Library/Python/3.7/lib/python/site-packages/urllib3/connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/Users/XujieSong/Library/Python/3.7/lib/python/site-packages/urllib3/util/connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/

AttributeError: 'NoneType' object has no attribute 'close'

In [41]:
sorted(result)[::-1][:10]

[(0.37900000000000006, 8, -0.05),
 (0.306, 6, -0.05),
 (0.29800000000000004, 20, -0.05),
 (0.29300000000000004, 12, -0.05),
 (0.292, 8, -0.06),
 (0.289, 10, -0.05),
 (0.2749999999999999, 28, -0.05),
 (0.2719999999999999, 26, -0.05),
 (0.263, 16, -0.05),
 (0.2560000000000001, 18, -0.05)]